# Defining tool schemas



For a model to be able to call tools, we need to pass in tool schemas that describe what the tool does and what it's arguments are. 

Chat models that support tool calling features implement a .bind_tools() method for passing tool schemas to the model. 

Tool schemas can be passed in as:

* Python functions (with typehints and docstrings)
* Pydantic models
* TypedDict classes
* LangChain Tool objects
  
  
Subsequent invocations of the model will pass in these tool schemas along with the prompt.

# Creating tools from functions
### @tool decorator
This @tool decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided.

In [1]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.


In [2]:
print(multiply.name)
print(multiply.description)
print(multiply.args)
multiply({'a':3,'b':5})

multiply
multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


15

In [5]:
@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


# Let's inspect some of the attributes associated with the tool.

@tool
def divide(a: int, b: int) -> int:
    """Perform division of a number"""
    return a/b


# Let's inspect some of the attributes associated with the tool.

@tool
def subtract(a: int, b: int) -> int:
    """Perform Subtraction"""
    return a-b


# Let's inspect some of the attributes associated with the tool.


In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

tools = [add, multiply,subtract,divide]
llm_with_tools = llm.bind_tools(tools)

In [7]:
llm_with_tools.invoke("What is 98/9")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rBILBD59dAEE5EaSzUKIBCo7', 'function': {'arguments': '{"a":98,"b":9}', 'name': 'divide'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 163, 'total_tokens': 180}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_db4a9208a8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7d0f6cc3-5d9a-4ba9-9e56-82c195aca45c-0', tool_calls=[{'name': 'divide', 'args': {'a': 98, 'b': 9}, 'id': 'call_rBILBD59dAEE5EaSzUKIBCo7'}])

In [8]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [9]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [10]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
agent_executor.invoke({"input": "What is 8 divided by 2!"})



> Entering new AgentExecutor chain...

Invoking: `divide` with `{'a': 8, 'b': 2}`


4.0The result of \( \frac{8}{2} \) is \( 4 \).

> Finished chain.


{'input': 'What is 8 divided by 2!',
 'output': 'The result of \\( \\frac{8}{2} \\) is \\( 4 \\).'}

In [12]:
agent_executor.invoke({"input": "What is 88*6!"})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 88, 'b': 720}`


63360The result of \( 88 \times 6! \) is 63,360.

> Finished chain.


{'input': 'What is 88*6!',
 'output': 'The result of \\( 88 \\times 6! \\) is 63,360.'}

In [13]:
agent_executor.invoke({"input": "if i have 3 apples and i get 567 more apples. How many apples will I have in all?"})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 3, 'b': 567}`


570If you have 3 apples and you get 567 more, you will have a total of 570 apples.

> Finished chain.


{'input': 'if i have 3 apples and i get 567 more apples. How many apples will I have in all?',
 'output': 'If you have 3 apples and you get 567 more, you will have a total of 570 apples.'}

In [14]:
agent_executor.invoke({"input": "Who is Elon Musk?"})



> Entering new AgentExecutor chain...
Elon Musk is a billionaire entrepreneur, inventor, and business magnate known for his role in founding and leading several high-profile technology companies. He was born on June 28, 1971, in Pretoria, South Africa. Musk is best known for his work with:

1. **Tesla, Inc.** - Musk is the CEO and product architect of Tesla, a company that specializes in electric vehicles, energy storage solutions, and solar energy products.

2. **SpaceX** - He founded SpaceX (Space Exploration Technologies Corp.) in 2002 with the goal of reducing space transportation costs and enabling the colonization of Mars. SpaceX has developed the Falcon and Starship rockets and the Dragon spacecraft.

3. **PayPal** - Musk co-founded X.com in 1999, which later became PayPal and was sold to eBay in 2002.

4. **Neuralink** - He co-founded Neuralink, a neurotechnology company focused on developing brain-computer interfaces.

5. **The Boring Company** - Musk founded this infrastruc

{'input': 'Who is Elon Musk?',
 'output': 'Elon Musk is a billionaire entrepreneur, inventor, and business magnate known for his role in founding and leading several high-profile technology companies. He was born on June 28, 1971, in Pretoria, South Africa. Musk is best known for his work with:\n\n1. **Tesla, Inc.** - Musk is the CEO and product architect of Tesla, a company that specializes in electric vehicles, energy storage solutions, and solar energy products.\n\n2. **SpaceX** - He founded SpaceX (Space Exploration Technologies Corp.) in 2002 with the goal of reducing space transportation costs and enabling the colonization of Mars. SpaceX has developed the Falcon and Starship rockets and the Dragon spacecraft.\n\n3. **PayPal** - Musk co-founded X.com in 1999, which later became PayPal and was sold to eBay in 2002.\n\n4. **Neuralink** - He co-founded Neuralink, a neurotechnology company focused on developing brain-computer interfaces.\n\n5. **The Boring Company** - Musk founded th